In [1]:
from Levenshtein import distance as levenshtein
from zipfile import ZipFile
from tqdm.notebook import tqdm, trange
import ncbi.datasets
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram
import os

In [4]:
# Plotting configuration
sns.set()
plt.rcParams.update({
    'text.usetex': False,
    'figure.constrained_layout.use': True,
    'figure.dpi': 300,
    'figure.figsize': ((W := 8.27 - 2*0.04167 - 2*(PAD := 0.2)), W/(3/2)),
    'savefig.pad_inches': 2*0.041672 + PAD,
    'savefig.bbox': 'tight',
    'font.size' : 11,
    'axes.labelsize': 11,
    'legend.fontsize': 11,
})
if plt.rcParams['text.usetex']:
    plt.rcParams.update({
        'text.latex.preamble': (
            r'\usepackage{lmodern}'
            r'\usepackage[T1]{fontenc}'
            r'\usepackage[utf8]{inputenc}'
            r'\usepackage{siunitx}'
            r'\usepackage{physics}'
            r'\usepackage{sansmath}'           # Disable for roman font
            r'\sansmath'                       # *
        ),
        # 'font.family': 'lmodern',              # Disable for sans-serif
    })
%config InlineBackend.figure_formats = 'png' #'svg', 'pdf', 'retina'

# Other config
if not 'chdir' in globals():
    os.chdir('..')
chdir = True

---

In [6]:
species = pd.read_csv('dat/data.csv')
species

,id,name,dna
0,GCA_905340225.1,Abrostola tripartita,AAAATCTTCCACCTGCTGAACATTCATATAATGAACTTCCAATTTT...
1,GCF_001949145.1,crown-of-thorns starfish,AGATAGAAACTGACCTGGCTCACGCCGGTCTGAACTCAGATCACGT...
2,GCA_016904835.1,rifleman,GTCCCTGTAGCTTAATAAAAAGCATAACACTGAAGATGTTAAGACG...
3,GCA_021347895.1,spiny chromis,GCGTAGCTTAACTAGAGCATAACTCTGAAGATGTTAGGGTGGACCT...
4,GCF_904848185.1,yellowfin seabream,GCTCACGTAGCTTAATTAAAGCATAACATTGAAGCCGTTAAGATGG...
...,...,...,...
719,GCA_020796205.1,Ziziphus jujuba var. spinosa,AGGGGCATCTTTCTTTCTCCCATTCTTTCTGTTGGTCAACAACCAA...
720,GCF_000696155.1,Zootermopsis nevadensis,AATGGAGTGCCTGAAAAAGGATTACTATGATAGGGTAAATAATGTA...
721,GCF_011800845.1,common lizard,GTCATTGTAGCTTATTTTTTTGAAAGCACGGCACTGAAAATGCCGA...
722,GCA_907165275.2,Zygaena filipendulae,ATTTAAAGCATTATTATTTATATGTGCTGGTGTTATTATCCATATA...


In [8]:
K_len = np.load('dat/K_len.npy')
K_lev = np.load('dat/K_lev.npy')